In [ ]:
import crispr as cr
from crispr.crispr_class import Crispr

kws_process_guide_rna = {"feature_split": "|", "guide_split": "-", 
                         "key_control_patterns": ["CTRL"]}
kws_umap = {"min_dist": 0.3}
kws_umap=kws_umap
kws_init = {"assay": None, "assay_protein": None, 
            "col_gene_symbols": "gene_symbols", 
            "col_cell_type": "majority_voting", 
            "col_sample_id": None, "col_batch": None, 
            "col_perturbed": "perturbation", 
            "col_guide_rna": "feature_call", 
            "col_num_umis": "num_umis", 
            "kws_process_guide_rna": kws_process_guide_rna,
            "col_condition": "target_gene_name", "key_control": "NT", 
            "key_treatment": "KD", "remove_multi_transfected": True}
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, None], "cell_filter_ncounts": [3, 40000], 
          "gene_filter_ncell": [3, None], "regress_out": None, 
          "kws_scale": "z", "kws_umap": kws_umap}

file_path = {'HH03': {'directory': 'data/crispr-screening/HH03'}, 
             'HH06': {'directory': 'data/crispr-screening/HH06'}}, 

In [ ]:
selves = [Crispr(file_path[f], **kws_init)for f in file_path]